In [1]:
import os

os.environ[
    "OMP_NUM_THREADS"
] = "128"  # for jupyter.nersc.gov otherwise the notebook only uses 2 cores

import matplotlib as mpl
mpl.rc('image', cmap='coolwarm')

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import numpy as np
import healpy as hp

In [3]:
from fgbuster.observation_helpers import get_noise_realization

In [4]:
from fgbuster import (CMB, Dust, Synchrotron, AnalyticComponent,
                      basic_comp_sep, 
                      get_observation, get_instrument)

In [5]:
from fgbuster.separation_recipes import harmonic_ilc as hilc


In [6]:
from plancklens.utils import camb_clfile
from lenspyx import synfast

In [7]:
import pysm3
import pysm3.units as u

In [8]:
from common import convert_units

In [248]:
noise_test = get_noise_realization(256, instrument)

In [251]:
nl = hp.anafast(noise_test[0])

In [254]:
np.var(noise_test[0][0])

26.76715185176525

In [256]:
def nlev2sig(nlev, nside):
    
    return np.sqrt((np.pi*nlev/10800)**2*(12*nside**2)/4/np.pi)

# simulations

In [10]:
cls_path = '/global/homes/j/jianyao/non_gau_lensing/theory/cls/'

In [250]:
instrument

,frequency,depth_p,depth_i,fwhm,f_sky,status,reference,type,note,experiment
0,27.0,100.4,71.0,7.4,0.4,forecast,Journal of Cosmology and Astroparticle Physics...,ground,depth_p is simply depth_i * sqrt(2),SO_LAT
1,39.0,50.9,36.0,5.1,0.4,forecast,Journal of Cosmology and Astroparticle Physics...,ground,depth_p is simply depth_i * sqrt(2),SO_LAT
2,93.0,11.3,8.0,2.2,0.4,forecast,Journal of Cosmology and Astroparticle Physics...,ground,depth_p is simply depth_i * sqrt(2),SO_LAT
3,145.0,14.1,10.0,1.4,0.4,forecast,Journal of Cosmology and Astroparticle Physics...,ground,depth_p is simply depth_i * sqrt(2),SO_LAT
4,225.0,31.1,22.0,1.0,0.4,forecast,Journal of Cosmology and Astroparticle Physics...,ground,depth_p is simply depth_i * sqrt(2),SO_LAT
5,280.0,76.4,54.0,0.9,0.4,forecast,Journal of Cosmology and Astroparticle Physics...,ground,depth_p is simply depth_i * sqrt(2),SO_LAT


In [15]:
class simulations:
    
    def __init__(self, nside, instrument):
        
        self.nside = nside
        self.instrument = instrument
        self.fres = instrument['frequency']
        self.fwhms = instrument['fwhm']

    def get_all(self, add_foreground = 'd0', noise = 'alms'):

        '''
        return one realization of (lensed CMB + foreground) with the beam applied + noise, 
        for each frequency defined in the instrument.
        
        noise: get noise alms or maps. 
        '''

        cl_unl = camb_clfile(os.path.join(cls_path, 'FFP10_wdipole_lenspotentialCls.dat'))
        geom_info = ('healpix', {'nside':nside}) # Geometry parametrized as above, this is the default

        lmax_unl = 2*nside
        cmb_temp = synfast(cl_unl, lmax=lmax_unl, verbose=0, geometry=geom_info, alm = False)
        self.cmb_len = np.row_stack((cmb_temp['T'], cmb_temp['QU'])) # (3, 12*nside**2)

        Nf = self.fres.size
        cmb_maps = np.repeat(self.cmb_len[np.newaxis, :, :], Nf, axis = 0)
             
        if add_foreground.startswith('d'):
            # for pysm3 pre-defined models
            foreground = get_observation(instrument, add_foreground, nside=nside, noise=False, unit='uK_CMB')
            
        elif add_foreground.startswith('forse'):
            
            pysm_model = add_foreground.split('_')[1]
            
            foreground = self.rescale_forse(self.fres, pysm_model = pysm_model)
            
        noise_maps = get_noise_realization(self.nside, instrument, unit='uK_CMB')
        noise_alms = []
        
        map_all = cmb_maps + foreground
        for i in range(Nf):
            # map_all[i] = pysm3.apply_smoothing_and_coord_transform(map_all[i], fwhm=self.fwhms[i]*u.arcmin) + self.noise_maps[i]
            map_all[i] = hp.smoothing(map_all[i], fwhm = self.fwhms[i]/180/60*np.pi) + noise_maps[i]
            foreground[i] = hp.smoothing(foreground[i], fwhm = self.fwhms[i]/180/60*np.pi)
            
            if noise == 'alms':
                noise_alms.append(hp.map2alm(noise_maps[i]))
                
        if noise == 'maps':
            self.noise_maps = noise_maps
        elif noise == 'alms':
            self.noise_alms = noise_alms
                
        self.map_all = map_all
        self.fg = foreground
        
    def rescale_forse(self, fres, pysm_model = 'd0'):
        '''
        Possible problem: hp.udgrade is used for nside changes; better choice:pysm3.apply_smoothing_and_coord_transform
        '''
                
        my_dust = self.model_forse(pysm_model)
            
        if fres.size > 1:
            _observations = np.zeros((len(fres), 3, 12*self.nside**2))
            for i in range(len(fres)):
                _observations[i] = my_dust.get_emission(fres[i] * u.GHz)*convert_units('uK_RJ', 'uK_CMB', fres[i])
                
        else:
            assert fres.size == 1
            _observations = my_dust.get_emission(fres * u.GHz)*convert_units('uK_RJ', 'uK_CMB', fres)
        
        return _observations
    
    def apply_hilc(self, components, lbins):
        
        '''
        apply fgbuster.hilc method to get clean CMB map
        '''
        
        results = hilc(components, self.instrument, self.map_all, lbins) 
        ### data are deconvolved when transforming to alms, 
        ### then using harmonic_ilc_alm to do the component separation
        weighted_alms = []
        
        noise_alms_ilc = np.copy(self.noise_alms)
        
        for i in range(self.fres.size):
            bl = hp.gauss_beam(np.radians(self.fwhms[i]/60.0), lmax = lbins[-1], pol=True)
            # all_alms = hp.map2alm(self.map_all[i], lmax = lbins[-1])
            fg_alms = hp.map2alm(self.fg[i], lmax = lbins[-1])
            
            for j in range(3): # T, E, B
                # hp.almxfl(all_alms[j], results.W[j, :, 0, i]/bl[:, j], inplace = True)  ### simulated total maps are deconvolved 
                hp.almxfl(fg_alms[j], results.W[j, :, 0, i]/bl[:, j], inplace = True)
                hp.almxfl(noise_alms_ilc[i][j], results.W[j, :, 0, i]/bl[:, j], inplace = True)  ### simulated ilc noise maps are not deconvolved during the component separation
                
            # weighted_alms.append(all_alms)
            weighted_alms.append(fg_alms)
            
        # cmb_alms = np.sum(weighted_alms, axis = 0)
        noise_ilc_alms = np.sum(noise_alms_ilc, axis = 0) 
        
        return results, noise_ilc_alms, fg_alms
    
    def model_forse(self, pysm_model):
        
        dust_dir = "/pscratch/sd/j/jianyao/data_lensing/processed_dust_maps/"
        if pysm_model == 'd0':
            my_dust = pysm3.ModifiedBlackBody(
                nside = self.nside,
                map_I = "pysm_2/dust_t_new.fits",
                map_Q = dust_dir + "forse_dust_Q_353GHz_deconvolved_lmax_4096_nside4096_uK_RJ.fits", #"forse_dust_Q_353GHz_3amin_nside4096_uK_RJ.fits",
                map_U = dust_dir + "forse_dust_U_353GHz_deconvolved_lmax_4096_nside4096_uK_RJ.fits",
                unit_I = "uK_RJ",
                unit_Q = "uK_RJ",
                unit_U = "uK_RJ",
                map_mbb_index = 1.54,
                map_mbb_temperature = 20,
                unit_mbb_temperature = "K",
                freq_ref_I = "545 GHz",
                freq_ref_P = "353 GHz"
            )
            
        if pysm_model == 'd1':
            my_dust = pysm3.ModifiedBlackBody(
                nside = self.nside,
                map_I = "pysm_2/dust_t_new.fits",
                map_Q = dust_dir + "forse_dust_Q_353GHz_deconvolved_lmax_4096_nside4096_uK_RJ.fits",
                map_U = dust_dir + "forse_dust_U_353GHz_deconvolved_lmax_4096_nside4096_uK_RJ.fits",
                unit_I = "uK_RJ",
                unit_Q = "uK_RJ",
                unit_U = "uK_RJ",
                map_mbb_index = "pysm_2/dust_beta.fits",
                map_mbb_temperature = "pysm_2/dust_temp.fits",
                unit_mbb_temperature = "K",
                freq_ref_I = "545 GHz",
                freq_ref_P = "353 GHz"
            )
            
        if pysm_model == 'd2':
            my_dust = pysm3.ModifiedBlackBody(
                nside = self.nside,
                map_I = "pysm_2/dust_t_new.fits",
                map_Q = dust_dir + "forse_dust_Q_353GHz_deconvolved_lmax_4096_nside4096_uK_RJ.fits",
                map_U = dust_dir + "forse_dust_U_353GHz_deconvolved_lmax_4096_nside4096_uK_RJ.fits",
                unit_I = "uK_RJ",
                unit_Q = "uK_RJ",
                unit_U = "uK_RJ",
                map_mbb_index = "pysm_2/beta_mean1p59_std0p2.fits",
                map_mbb_temperature = "pysm_2/dust_temp.fits",
                unit_mbb_temperature = "K",
                freq_ref_I = "545 GHz",
                freq_ref_P = "353 GHz"
            )
            
        return my_dust

In [16]:
nside = 1024
instrument = get_instrument('SO_LAT')


# add_foreground = 'forse3_d1'

# cases = ['d0', 'd1', 'forse3_d0', 'forse3_d1']
cases = ['forse3_d0']
for add_foreground in cases:
    
    for mc in range(20):
        sims = simulations(nside, instrument)
        sims.get_all(add_foreground = add_foreground, noise = 'alms')

        components = [CMB()]
        lbins = np.arange(42)*50

        results, noise_ilc_alms, fg_res_alms = sims.apply_hilc(components, lbins)
        hp.write_map('/pscratch/sd/j/jianyao/data_lensing/cleaned_CMB/SO_LAT/CMB_map_from_SO_LAT_%s_HILC_lbins_42x50_lmax_2050_nside_1024_%04d.fits'%(add_foreground, mc), results.s[0], overwrite = True)
        hp.write_alm('/pscratch/sd/j/jianyao/data_lensing/cleaned_CMB/SO_LAT/Noise_ilc_alms_from_SO_LAT_%s_HILC_lbins_42x50_lmax_2050_nside_1024_%04d.fits'%(add_foreground, mc), noise_ilc_alms, overwrite = True)
        hp.write_alm('/pscratch/sd/j/jianyao/data_lensing/cleaned_CMB/SO_LAT/FG_ilc_alms_from_SO_LAT_%s_HILC_lbins_42x50_lmax_2050_nside_1024_%04d.fits'%(add_foreground, mc), fg_res_alms, overwrite = True)
        

        # plt.figure(figsize = (10, 8))
        # s = 0; e = lbins[-1]
        # coeff = ells*(ells+1)/2/np.pi #*coeff
        # plt.plot(results['cl_out'][0][0][s:e], '--', label = 'TT')
        # plt.loglog(results['cl_out'][0][1][s:e], '--', label = 'EE')
        # plt.loglog(results['cl_out'][0][2][s:e], '--', label = 'BB')
        # plt.loglog(nl[0])
        # plt.loglog(nl[1])
        # plt.loglog(nl[2])